Source: Kaggle -https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud


In [29]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import tkinter as tk
from tkinter import messagebox

In [31]:
from sklearn.datasets import make_classification

df = pd.read_csv('creditcard.csv')

In [33]:
X, y = make_classification(n_samples=1000, n_features=2, n_informative=2, 
                           n_redundant=0, n_clusters_per_class=1, 
                           weights=[0.99, 0.01], random_state=42)

# Create a DataFrame for convenience
data = pd.DataFrame(X, columns=["Amount", "Time"])
data["Fraud"] = y

# Check the first few rows of the dataset
print(data.head())

     Amount      Time  Fraud
0  1.073546 -1.101339      0
1  0.755945 -1.172352      0
2  1.354479 -0.948528      0
3  3.103090  0.233485      0
4  1.951888 -0.270157      0


In [34]:
X_train, X_test, y_train, y_test = train_test_split(
    data[["Amount", "Time"]], data["Fraud"], test_size=0.2, random_state=42
)

In [35]:

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [36]:
# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)


In [37]:
# Train a Random Forest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier(random_state=42)

In [38]:
# UI for Fraud Detection
def test_fraud_detection():
    try:
        # Get user inputs
        amount = float(entry_amount.get())
        time = float(entry_time.get())

        # Create input data with correct feature names
        input_data = pd.DataFrame([[amount, time]], columns=["Amount", "Time"])

        # Standardize the input
        input_data_scaled = scaler.transform(input_data)

        # Predict if the transaction is fraudulent
        prediction = model.predict(input_data_scaled)

        # Show the result
        if prediction[0] == 1:
            messagebox.showinfo("Result", "This transaction is fraudulent!")
        else:
            messagebox.showinfo("Result", "This transaction is legitimate.")
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {e}")

In [39]:
# Tkinter UI
app = tk.Tk()
app.title("Fraud Detection System")


''

In [40]:
# Labels and Inputs
tk.Label(app, text="Enter Amount:").grid(row=0, column=0, padx=10, pady=10)
entry_amount = tk.Entry(app)
entry_amount.grid(row=0, column=1, padx=10, pady=10)

tk.Label(app, text="Enter Time:").grid(row=1, column=0, padx=10, pady=10)
entry_time = tk.Entry(app)
entry_time.grid(row=1, column=1, padx=10, pady=10)


In [41]:
# Button to Test Fraud Detection
btn_test = tk.Button(app, text="Check Transaction", command=test_fraud_detection)
btn_test.grid(row=2, column=0, columnspan=2, pady=20)

# Run the App
app.mainloop()